## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Изучение общей информации

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
data[data.days_employed.isnull()] #показывается список "days_employed" NaN
display(data.head(20))

#дополнительные операциия для анализа общей информации

#nan_incope_type = data.groupby('income_type')['total_income'].isnull().count()
#data_nan = data['total_income'].duplicated()
#print(data['total_income'].duplicated())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

В столбцах 'days_employed' и 'total_income' отсутствуют данные и их количества равны. Столбец 'education' приведен в нижнем и верхнем регистре, столбец 'days_employed' приведен в отрицательных числах и в другой размерности.
Последовательность действий:
    1. Обработка столбца 'days_employed' - положительные значения, перевод в другую единицу измерения.
    2. Группирровка данных для 'total_income' по типу занятисти.


### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#Замена пропущенных данных в столбцах 'days_employed' и 'total_income'
#на медиану соответсвующего столбца

#1. Обработка столбца 'days_employed' - положительные значения, 
#перевод в другую единицу измерения.

def days_to_years(row): #функция для перевода из часа в года
    years = row['days_employed']/365
    if years > row['dob_years']:
        return years/24
    return years

data['days_employed'] = abs(data['days_employed'])
data['days_employed'] = data.apply(days_to_years, axis = 1)
display(data.head(20)) 


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,38.843159,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,2.537495,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,7.888225,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,0.418574,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,18.985932,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,5.996593,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
  
# 2. Замена пропусков в'days_employed' и 'total_income' на медиану по группам

data['days_employed'] = data['days_employed'].fillna(data.groupby('income_type')['days_employed'].transform('median'))
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))



# вывод датафрейма для визуальной проверки
data.info()
display(data.head(50))



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,38.843159,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,2.537495,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,7.888225,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,0.418574,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,18.985932,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,5.996593,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Выполнена замена пропусков в'days_employed' и 'total_income' на медиану

### Замена типа данных

In [4]:
# Замена типа данных в в'days_employed' и 'total_income'

data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

data.info()

# вывод датафрейма для визуальной проверки

#display(data.head(20))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Выполнена замена типа данных в'days_employed' и 'total_income'

### Обработка дубликатов

In [12]:

#print('Размер таблицы до обработки дубликатов', data.shape)

data['education'] = data['education'].str.lower()

#print(data.duplicated().sum())

data = data.drop_duplicates().reset_index(drop = True)

#print('Размер таблицы после обработки дубликатов', data.shape)
#print(data.duplicated().sum())

# хотел проверить удаляет ли drop_duplicates() все строчку или хотя бы одну оставляет, но так и не понял(

### Вывод

В датафрейме было 71 строки дубликатов, которые дропнулись.

### Лемматизация

In [6]:
from pymystem3 import Mystem
m = Mystem()

def lemma(row):
    lemma = m.lemmatize(row)
    lemma = ''.join(lemma)
    return lemma.rstrip('\n')

data['purpose'] = data['purpose'].apply(lemma)
sum = data.groupby('purpose').count()

#Проверка лемматизации
#print(sum)

data.head(50)
                 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилье
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиль
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилье
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительный образование
4,0,38,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьба
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилье
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операция с жилье
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьба
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилье для семья


### Вывод

Выполнена лемматизация столбца 'purpose' для работой с ней


### Категоризация данных

In [7]:
#data['purpose'].unique()
# Из писка видно, что люди планируют покупки по следующим категориям:
# 1. жилье(недвижимость) 2. автомобиль 3. образование 4. свадьба

def proverka_1(row):
    counts = 0
    if 'автомобиль' in row:
        counts += 1
        return counts 
    
def proverka_2(row):
    counts = 0
    if 'образование' in row:
        counts += 1
        return counts  
    
def proverka_3(row):
    counts = 0
    if 'свадьба' in row:
        counts += 1
        return counts 
    
def proverka_4(row):
    counts = 0
    if 'жилье' in row:
        counts += 1
        return counts 
    
def proverka_5(row):
    counts = 0
    if 'недвижимость' in row:
        counts += 1
        return counts 
    
sum_1 = data['purpose'].apply(proverka_1).sum()
sum_2 = data['purpose'].apply(proverka_2).sum()
sum_3 = data['purpose'].apply(proverka_3).sum()
sum_4 = data['purpose'].apply(proverka_4).sum()
sum_5 = data['purpose'].apply(proverka_5).sum()
sum = sum_1 + sum_2 + sum_3 + sum_4 + sum_5

# Окончание проверки данных перед категоризацией


# Создадим фукцию для распределения запланируемых покупок на 4 категории

def categorization(row):
    if 'автомобиль' in row:
        return 'автомобиль'    
    elif 'образование' in row:
        return 'образование'
    elif 'свадьба' in row:
        return 'свадьба'    
    elif 'жилье' or 'недвижимость' in row:
        return 'жилье'
    elif 'недвижимость' in row:
        return 'жилье' 
    else:
        return row

data['purpose'] = data['purpose'].apply(categorization)

# сравнение категоризацией данных с проверкой
display(data['purpose'].value_counts()) 

#data.head(50)



жилье          10811
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose, dtype: int64

### Вывод

Большинство людей берут кредит на жилье, почти в равных долях берут кредит на автомобиль и образование, а кто хочет сделать пышную свадьбу, тоже не отказываются от кредита. 

### Шаг 3. Определние зависимостей

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
def children(row):
    witt_children = row['children']
    if witt_children <= 0:
        return "без детей"
    elif witt_children > 2:
        return "многодетная"
    return "1-2 ребенка"

data['children'] = data.apply(children, axis = 1)
#display(data['children'])    
dta_1 = data.groupby('children').agg({'debt': ['count', 'sum', 'mean']})
dta_1['percent'] = dta_1['debt']['mean']*100
dta_1.columns = ['_'.join(col) for col in dta_1.columns.values]
dta_1 = dta_1.reset_index()
display(dta_1)

,children,debt_count,debt_sum,debt_mean,percent_
0,1-2 ребенка,6860,638,0.093003,9.300292
1,без детей,14138,1064,0.075258,7.525817
2,многодетная,456,39,0.085526,8.552632


### Вывод

В основном бездетные семьи охотнее берут кредит в банке, потому что семейный бюджет не включен затраты на обеспечение детей. У бездетных семей самая низкий возрат кредита в срок.
С увеличением количества детей, уменьшается количество семей.


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
dta_2 = data.groupby('family_status').agg({'debt': ['count', 'sum', 'mean']})
dta_2['percent'] = dta_2['debt']['mean']*100
dta_2.columns = ['_'.join(col) for col in dta_2.columns.values]
dta_2 = dta_2.reset_index()
display(dta_2)

,family_status,debt_count,debt_sum,debt_mean,percent_
0,Не женат / не замужем,2810,274,0.097509,9.750890
1,в разводе,1195,85,0.071130,7.112971
2,вдовец / вдова,959,63,0.065693,6.569343
3,гражданский брак,4151,388,0.093471,9.347145
4,женат / замужем,12339,931,0.075452,7.545182


### Вывод

Большего всего берут кредит семье, которые состоят в официальном браке ("женат / замужем").
Семейные положения такие, как "Не женат / не замужем" и "гражданский брак" предрасположены к нарушениям выплат кредита.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
#для распределения ежемесячного дохода по категориям, используем данные 
# из статьи https://vc.ru/finance/128505 "Самая распространенная зарплата в России..."

def distribution_income(row):
    
    income = row['total_income']
    
    if income <= 11280:
        return "ниже МРОТ"
    elif 0 < income <= 24800:
        return '1) от 11.3 тыс.₽ до 24.8 тыс.₽'
    elif 0 < income <= 50000:
        return '2) от 24.8 тыс.₽ до 50 тыс.₽'
    elif 50000 < income <= 75000:
        return '3) от 50 тыс.₽ до 75 тыс.₽'
    elif 75000 < income <= 100000:
        return '4) от 75 тыс.₽ до 100 тыс.₽'
    elif 100000 < income <= 250000:
        return '5) от 100 тыс.₽ до 250 тыс.₽'
    elif 250000 < income <= 500000:
        return '6) от 250тыс.₽ до 500 тыс.₽'
    elif income > 500000:
        return '7) от 500 тыс.₽ и выше'
    

data['total_income_dist'] = data.apply(distribution_income, axis = 1)
#display(data['total_income_dist'])    

data.sort_values(by = ('total_income_dist'))

dta_3 = data.groupby('total_income_dist').agg({'debt': ['count', 'sum', 'mean']})
dta_3['percent'] = dta_3['debt']['mean']*100
dta_3.columns = ['_'.join(col) for col in dta_3.columns.values]
dta_3.reset_index()

display(dta_3)



,debt_count,debt_sum,debt_mean,percent_
total_income_dist,,,,
1) от 11.3 тыс.₽ до 24.8 тыс.₽,8,1,0.125000,12.500000
2) от 24.8 тыс.₽ до 50 тыс.₽,364,22,0.060440,6.043956
3) от 50 тыс.₽ до 75 тыс.₽,1493,113,0.075687,7.568654
4) от 75 тыс.₽ до 100 тыс.₽,2598,218,0.083911,8.391070
5) от 100 тыс.₽ до 250 тыс.₽,14178,1193,0.084144,8.414445
6) от 250тыс.₽ до 500 тыс.₽,2591,180,0.069471,6.947125
7) от 500 тыс.₽ и выше,222,14,0.063063,6.306306


### Вывод

Большего всего берут кредит семье, у которых доход "от 100 тыс.₽ до 250 тыс.₽". 

- Как разные цели кредита влияют на его возврат в срок?

In [11]:
# 1. Метод №1 - категоризация с groupby

#dta_4 = data.groupby('purpose').agg({'debt': ['count', 'sum', 'mean']})
#dta_4['percent'] = dta_4['debt']['mean']*100
#dta_4.columns = ['_'.join(col) for col in dta_4.columns.values]
#dta_4 = dta_4.reset_index()

#display(dta_4)


# 2. Метод №2 - категоризация с pivot_table()

table = data.pivot_table(values='debt', index='purpose', aggfunc=['count', 'sum', 'mean'] )
table['percent'] = table['mean']['debt']*100
table.columns = ['_'.join(col) for col in table.columns.values]
table = table.reset_index()
display(table)


,purpose,count_debt,sum_debt,mean_debt,percent_
0,автомобиль,4306,403,0.093590,9.359034
1,жилье,10811,782,0.072334,7.233373
2,образование,4013,370,0.092200,9.220035
3,свадьба,2324,186,0.080034,8.003442


### Вывод

Большего всего берут кредит на жилье и имеют низкий процент нарушения выплат кредита.

### Шаг 4. Общий вывод

В столбцах 'days_employed' и 'total_income' отсутствуют данные и их количества равны. Столбец 'education' приведен в нижнем и верхнем регистре, столбец 'days_employed' приведен в отрицательных числах и в другой размерности. Последовательность действий:

1. Обработка столбца 'days_employed' - положительные значения, перевод в другую единицу измерения.
2. Группирровка данных для 'total_income' по типу занятисти.

Выполнена замена пропусков в'days_employed' и 'total_income' на медиану. Данные в 'days_employed' преведены в положительные значения и вединицу измерения "год". В датафрейме проведена очистка 71 строки дубликатов, также выполнена лемматизация столбца 'purpose'.

В основном бездетные семьи охотнее берут кредит в банке, потому что семейный бюджет не включен затраты на обеспечение детей. У бездетных семей самая низкий возрат кредита в срок.
С увеличением количества детей, уменьшается количество семей.
Процент должников в зависимости от количества детей, имеют следующия значения:
 - бездетная семья - 7.56%
 - 1-2 ребенка в семье - 9.30%
 - многодетная семья - 8.55%

Большего всего берут кредит семье, которые состоят в официальном браке ("женат / замужем"). Семейные положения такие, как "Не женат / не замужем" и "гражданский брак" предрасположены к нарушениям выплат кредита.
Процент должников в зависимости от семейного положения, имеют следующия значения:
 - не женат / не замужем - 9.75%
 - в разводе - 7.11%
 - вдовец / вдова - 6.57%
 - гражданский брак - 9.35%
 - женат / замужем - 7.55%

Большего всего берут кредит семье, у которых доход "от 100 тыс.₽ до 250 тыс.₽". У людей с доходом от 24.8 тыс.₽ до 50 тыс.₽ низкий процент нарушения выплат кредита.
Процент должников в зависимости от ежемесячного дохода, имеют следующия значения:
 - от 11.3 тыс.₽ до 24.8 тыс.₽ - 12.50%
 - от 24.8 тыс.₽ до 50 тыс.₽ - 6.04%
 - от 50 тыс.₽ до 75 тыс.₽ - 7.57%
 - от 75 тыс.₽ до 100 тыс.₽ - 8.39%
 - от 100 тыс.₽ до 250 тыс.₽- 8.41%
 - от 250тыс.₽ до 500 тыс.₽ - 6.95%
 - от 500 тыс.₽ и выше - 6.31%

Большего всего берут кредит на жилье и имеют низкий процент нарушения выплат кредита.
Процент должников в зависимости от целей кредита, имеют следующия значения:
 - автомобиль - 9.36%
 - жилье - 7.23%
 - образование - 9.22%
 - свадьба - 8.0%

Портрет идеального заемщика соответствует следующим позициям:
 - Бездетная семья;
 - Семейное положение - "женат / замужем"
 - Доход - "от 0 тыс.₽ до 50 тыс.₽" или "от 500 тыс.₽ и выше	"
 - Цель кредита - "жилье"	
 